In [29]:
import gymnasium as gym
from stable_baselines3 import PPO
from gym_chrono.envs.wheeled.off_road_artACL import off_road_art
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.tensorboard import SummaryWriter

def init_trajectory_plot(bmp_image_path):
    bmp_image = Image.open(bmp_image_path)
    bmp_array = np.array(bmp_image)

    fig, ax = plt.subplots(figsize=(12, 12), dpi=300)
    ax.imshow(bmp_array, cmap='gray', alpha=1)
    ax.set_title("Vehicle Trajectories on Test Terrain", fontsize=20, color='white', fontweight='bold')
    
    # Remove X and Y labels
    ax.set_xticks([])
    ax.set_yticks([])

    # Set the figure background to black for better contrast
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    return fig, ax

def update_trajectory_plot(ax, trajectory, trial):
    x_coords, y_coords = zip(*trajectory)
    
    # Increase line thickness and transparency
    ax.plot(x_coords, y_coords, linewidth=1.5, alpha=0.6, color='white')  # Thicker lines with more opacity
    
    start = trajectory[0]
    end = trajectory[-1]
    
    # Solid markers for start and end (filled dot)
    ax.plot(start[0], start[1], 'o', markersize=8, markeredgecolor='white', markerfacecolor='white', alpha=0.9)  # Solid dot for start
    ax.plot(end[0], end[1], 'o', markersize=8, markeredgecolor='white', markerfacecolor='white', alpha=0.9)    # Solid dot for end

    # Update the title to show the current trial number with a bold font
    ax.set_title(f"50 Evaluation Trajectories on Test Terrain", fontsize=20, color='white', fontweight='bold')


def save_trajectory_plot(fig, output_path):
    # Tight layout to ensure the plot saves properly
    plt.tight_layout()

    # Save the figure
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='black', edgecolor='none')
    print(f"Saved trajectory visualization to {output_path}")

def evaluate_model(env, model, num_trials=50, max_steps=1000, render=False, bmp_image_path=None):
    all_trajectories = []
    success_count = 0
    traversal_times = []
    episode_roll_means = []
    episode_pitch_means = []

    # Initialize the plot
    fig, ax = init_trajectory_plot(bmp_image_path)

    for trial in range(num_trials):
        obs, _ = env.reset(seed=trial)
        if render:
            env.render('follow')

        step_count = 0
        roll_angles = []
        pitch_angles = []
        trajectory = []

        while step_count < max_steps:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            if render:
                env.render('follow')

            # Get the vehicle's position on the bitmap image
            pos_bmp_x, pos_bmp_y = env.get_vehicle_position_on_bmp()
            trajectory.append((pos_bmp_x, pos_bmp_y))

            step_count += 1

            # Collect roll and pitch angles at each step
            euler_angles = env.m_vehicle.GetVehicle().GetRot().Q_to_Euler123()
            roll_angles.append(np.degrees(abs(euler_angles.x)))
            pitch_angles.append(np.degrees(abs(euler_angles.y)))

            if terminated or truncated:
                if terminated and step_count < 200:  # Successful trial
                    success_count += 1
                    traversal_times.append(env.m_system.GetChTime())
                break

        all_trajectories.append(trajectory)

        # Update the plot with the new trajectory
        update_trajectory_plot(ax, trajectory, trial)

        # Save the updated plot after each trial
        save_trajectory_plot(fig, os.path.join("./PLRTest_logs/TrajEval/iter102_level18Final", f"traj{trial+1}.png"))

        # Calculate mean roll and pitch angles for this episode
        episode_roll_means.append(np.mean(roll_angles))
        episode_pitch_means.append(np.mean(pitch_angles))

    plt.close(fig)

    mean_traversal_time = np.mean(traversal_times) if traversal_times else 0
    var_traversal_time = np.var(traversal_times) if traversal_times else 0
    avg_roll_angle = np.mean(episode_roll_means)
    var_roll_angle = np.var(episode_roll_means)
    avg_pitch_angle = np.mean(episode_pitch_means)
    var_pitch_angle = np.var(episode_pitch_means)

    return all_trajectories, success_count, mean_traversal_time, var_traversal_time, avg_roll_angle, var_roll_angle, avg_pitch_angle, var_pitch_angle

def evaluate_single_model(env, model_file, test_level_dir, num_trials=50, max_steps=1000, render=False, writer=None):
    level_file = "level_5.bmp"
    level_path = os.path.join(test_level_dir, level_file)
    env.terrain_file = [level_path]
    env.update_terrain_stage(level_index=0)

    # Reset the environment to ensure everything is initialized
    env.reset()

    # Now we can safely access the bitmap_file
    actual_terrain_path = env.bitmap_file

    # Load the single model for evaluation
    model = PPO.load(model_file)

    # Evaluate the model
    all_trajectories, success_count, mean_traversal_time, var_traversal_time, avg_roll_angle, var_roll_angle, avg_pitch_angle, var_pitch_angle = evaluate_model(
        env, model, num_trials=num_trials, max_steps=max_steps, render=render, bmp_image_path=actual_terrain_path)

    success_rate = float(success_count) / num_trials

    # Log the success rate for the test level
    if writer is not None:
        writer.add_scalar(f'Success Rate/Test Level 5', success_rate, 0)  # Iteration 0 for a single model

    print(f"Test Level {level_file}: Success rate = {success_rate}\n")
    print(f"Mean Traversal Time: {mean_traversal_time:.2f} s")
    print(f"Average Roll/Pitch Angle: {avg_roll_angle:.2f}/{avg_pitch_angle:.2f} deg")
    print(f"Variance Roll/Pitch Angle: {var_roll_angle:.2f}/{var_pitch_angle:.2f} deg")

    return success_rate, mean_traversal_time, var_traversal_time, avg_roll_angle, var_roll_angle, avg_pitch_angle, var_pitch_angle

# Main function to run the simulation and evaluate the specific model
if __name__ == "__main__":
    checkpoint_dir = '../train/logs/vws_ppo_checkpoints_PLRNorm/'
    model_file = os.path.join(checkpoint_dir, "ppo_checkpoint_iter102_level18")  # Use this specific model
    test_level_dir = '../TestLevels'
    
    # Set up TensorBoard writer for logging
    log_dir = f'./PLRTest_logs/TrajEval'
    writer = SummaryWriter(log_dir)

    # Create the custom off-road environment
    env = off_road_art()
    
    # Evaluate the single model on the test level for 50 trials
    evaluate_single_model(env, model_file, test_level_dir, num_trials=50, max_steps=1000, render=False, writer=writer)

    writer.close()

Terrain loaded from /home/tong/Documents/gym_chrono/envs/wheeled/../data/terrain_bitmaps/Automatic-CL/TrainLevels/../TestLevels/level_5.bmp
warning cbtCollisionDispatcher::needsCollision: static-static collision!
--------------------------------------------------------------
Goal Reached
Initial position:  [ 10.3561, 6.3581, 3 ]
Goal position:  [ -10.4279, 1.22851, 2.5 ]
--------------------------------------------------------------
Saved trajectory visualization to ./PLRTest_logs/TrajEval/iter102_level18Final/traj1.png
warning cbtCollisionDispatcher::needsCollision: static-static collision!
--------------------------------------------------------------
Goal Reached
Initial position:  [ 7.1447, 9.15326, 3 ]
Goal position:  [ -10.0298, -3.12348, 3 ]
--------------------------------------------------------------
Saved trajectory visualization to ./PLRTest_logs/TrajEval/iter102_level18Final/traj2.png
warning cbtCollisionDispatcher::needsCollision: static-static collision!
----------------